# Residual Net for the CIFAR-10 dataset

In this notebook, I will implement a residual network of 20 layers, following the architecture from the paper :
- Kaiming He et al., 2015, Deep Residual Learning for Image Recognition

The main novel from these residual networks is that they add shortcut connection that let the model perform the identity transformation easily, if necessary. This, in theory, allows the residual network to only improve or keep the same performance for each layer added, while other networks see their performance decline after a certain threshold of depth.

In [1]:
import numpy as np
import keras
from keras.layers import Input, Activation, Conv2D, AvgPool2D
from keras.layers import Dense, Flatten, BatchNormalization
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.datasets import cifar10
from keras.callbacks import ModelCheckpoint
import time

/home/arthur/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Data Preparation

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

print('X shape :', x_train.shape)
print(len(x_train), 'train samples')
print(len(x_test), 'test samples')

X shape : (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
x_train_mean = np.mean(x_train, axis=0)
x_train_std = np.std(x_train, axis=0)

x_train = (x_train - x_train_mean)/x_train_std
x_test = (x_test - x_train_mean)/x_train_std

n_y = 10
y_train = keras.utils.to_categorical(y_train, n_y)
y_test = keras.utils.to_categorical(y_test, n_y)

# Model

In [4]:
l2_reg = keras.regularizers.l2(0.0001)

def residual_unit(x_input, filters, stride=1, regularizer=l2_reg):
    x = Conv2D(filters, kernel_size=(3,3), padding='same', use_bias=False, 
               strides=stride, kernel_regularizer=regularizer)(x_input)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Conv2D(filters, kernel_size=(3,3), padding='same', use_bias=False, 
               kernel_regularizer=regularizer)(x)
    x = BatchNormalization()(x)
    
    if stride!=1:
        x_input = AvgPool2D(pool_size=stride)(x_input)
        x_input = Conv2D(filters, kernel_size=(1,1), use_bias=False, 
                         kernel_regularizer=regularizer)(x_input)
        x_input = BatchNormalization()(x_input)
    
    x_res = keras.layers.add([x, x_input])
    
    x_out = Activation('relu')(x_res)
    
    return x_out

In [5]:
x_input = Input(shape=(32, 32, 3))

x = Conv2D(16, (3,3), padding='same', use_bias=False, 
           kernel_regularizer=l2_reg)(x_input)
x = BatchNormalization()(x)
x = Activation('relu')(x)

x = residual_unit(x, 16)
x = residual_unit(x, 16)
x = residual_unit(x, 16)

x = residual_unit(x, 32, stride=2)
x = residual_unit(x, 32)
x = residual_unit(x, 32)

x = residual_unit(x, 64, stride=2)
x = residual_unit(x, 64)
x = residual_unit(x, 64)

x = AvgPool2D((8,8))(x)
x = Flatten()(x)
output = Dense(10, activation='softmax')(x)

model = Model(x_input, output)

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 16)   432         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 16)   64          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 16)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

# Model training

In [6]:
# Set optimizer to train the model
optimizer = keras.optimizers.SGD(0.1, momentum=0.9, nesterov=True)
model.compile(optimizer, keras.losses.categorical_crossentropy, ['accuracy'])

# Data augmentation
shift = 4/32
generator = ImageDataGenerator(width_shift_range=shift, height_shift_range=shift, 
                               horizontal_flip=True)

batch_size = 64
n_steps = x_train.shape[0]//batch_size # training steps per epoch


save_path = './Model_trained/ResNet20_cifar10.h5'
ckeckpoint = ModelCheckpoint(save_path, monitor='val_acc', save_best_only=True, verbose=1)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=np.sqrt(0.1), patience=5, 
                                              min_delta=0.01, min_lr=1e-6)
early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=20)

t0 = time.time()
model.fit_generator(generator.flow(x_train, y_train, batch_size=batch_size), steps_per_epoch=n_steps, 
                    epochs=200, validation_data=(x_test, y_test), 
                    callbacks=[ckeckpoint, reduce_lr, early_stopping])
print('Total training time : %.3f s' %(time.time()-t0))

Epoch 1/200
781/781 [==============================] - 32s 40ms/step - loss: 1.7758 - acc: 0.3718 - val_loss: 2.7454 - val_acc: 0.3199

Epoch 00001: val_acc improved from -inf to 0.31990, saving model to ./Model_trained/ResNet20_cifar10.h5
Epoch 2/200
781/781 [==============================] - 28s 36ms/step - loss: 1.2362 - acc: 0.5947 - val_loss: 3.7490 - val_acc: 0.3501

Epoch 00002: val_acc improved from 0.31990 to 0.35010, saving model to ./Model_trained/ResNet20_cifar10.h5
Epoch 3/200
781/781 [==============================] - 27s 34ms/step - loss: 1.0049 - acc: 0.6912 - val_loss: 1.2808 - val_acc: 0.6170

Epoch 00003: val_acc improved from 0.35010 to 0.61700, saving model to ./Model_trained/ResNet20_cifar10.h5
Epoch 4/200
781/781 [==============================] - 27s 34ms/step - loss: 0.8913 - acc: 0.7408 - val_loss: 1.1801 - val_acc: 0.6808

Epoch 00004: val_acc improved from 0.61700 to 0.68080, saving model to ./Model_trained/ResNet20_cifar10.h5
Epoch 5/200
781/781 [==========


Epoch 00041: val_acc did not improve from 0.89930
Epoch 42/200
781/781 [==============================] - 27s 35ms/step - loss: 0.2967 - acc: 0.9504 - val_loss: 0.5150 - val_acc: 0.8883

Epoch 00042: val_acc did not improve from 0.89930
Epoch 43/200
781/781 [==============================] - 28s 36ms/step - loss: 0.2894 - acc: 0.9511 - val_loss: 0.4860 - val_acc: 0.8984

Epoch 00043: val_acc did not improve from 0.89930
Epoch 44/200
781/781 [==============================] - 28s 36ms/step - loss: 0.2856 - acc: 0.9517 - val_loss: 0.4881 - val_acc: 0.8943

Epoch 00044: val_acc did not improve from 0.89930
Epoch 45/200
781/781 [==============================] - 28s 36ms/step - loss: 0.2828 - acc: 0.9514 - val_loss: 0.5230 - val_acc: 0.8887

Epoch 00045: val_acc did not improve from 0.89930
Epoch 46/200
781/781 [==============================] - 27s 35ms/step - loss: 0.2778 - acc: 0.9517 - val_loss: 0.5242 - val_acc: 0.8893

Epoch 00046: val_acc did not improve from 0.89930
Epoch 47/200
7


Epoch 00083: val_acc did not improve from 0.91000
Epoch 84/200
781/781 [==============================] - 26s 33ms/step - loss: 0.1650 - acc: 0.9832 - val_loss: 0.4465 - val_acc: 0.9086

Epoch 00084: val_acc did not improve from 0.91000
Epoch 85/200
781/781 [==============================] - 26s 33ms/step - loss: 0.1642 - acc: 0.9841 - val_loss: 0.4464 - val_acc: 0.9094

Epoch 00085: val_acc did not improve from 0.91000
Epoch 86/200
781/781 [==============================] - 26s 33ms/step - loss: 0.1643 - acc: 0.9837 - val_loss: 0.4472 - val_acc: 0.9095

Epoch 00086: val_acc did not improve from 0.91000
Epoch 87/200
781/781 [==============================] - 26s 33ms/step - loss: 0.1634 - acc: 0.9843 - val_loss: 0.4463 - val_acc: 0.9096

Epoch 00087: val_acc did not improve from 0.91000
Epoch 88/200
781/781 [==============================] - 26s 34ms/step - loss: 0.1630 - acc: 0.9843 - val_loss: 0.4466 - val_acc: 0.9094

Epoch 00088: val_acc did not improve from 0.91000
Total training

# Final evaluation

In [8]:
best_model = keras.models.load_model(save_path)
print('Best model test accuracy :', best_model.evaluate(x_test, y_test, batch_size=64)[1])

10000/10000 [==============================] - 3s 304us/step
Best model test accuracy : 0.91


# Conclusion

This Residual network of 20 layers is, despite his depth, quite simple in his architecture. The shortcut connection, the vital part of the residual block, let us train a deep (and thin) network, able to score $91 \%$ accuracy on the CIFAR-10 dataset. This result is in line with the result from the original paper *Deep Residual Learning for Image Recognition*, their 20 layers model reach a very close $91.25 \%$ accuracy.